In [78]:
import xlrd
import os
import re
from gensim import corpora, models
import jieba.posseg as jp
import jieba
import spacy
nlp = spacy.load("en_core_web_sm")

In [3]:
path='data/'
All_notes=[]

In [5]:
for filename in os.listdir(path):
    xlsx = xlrd.open_workbook(path+filename)
    sheet1 = xlsx.sheets()[0]
    for i in range(6,sheet1.nrows-1):
        All_notes.append(sheet1.row_values(i)[-2])

In [8]:
spa_notes=[nlp(note) for note in All_notes]

In [9]:
sents_notes=[[i for i in note.sents] for note in spa_notes]

In [59]:
sents_notes[1][0][1].pos_=='NOUN'

True

In [79]:
nltk.pos_tag("go")

[('g', 'NN'), ('o', 'NN')]

In [80]:
token_notes=[]
for note in sents_notes:
    nt = []
    for sent in note:
        st = []
        for token in sent:
            if nlp.vocab[token.lemma_].is_stop==False:
                st.append(token.lemma_.lower())
        nt.append(st.copy())
    token_notes.append(nt.copy())

In [81]:
split_tokens=[]
for i in token_notes:
    split_tokens+=i

In [82]:
r1 = u'[0-9’!"#$%&\' \n()*+,-./:;<=>?@，。?★、…【】《》？“”‘’！[\\]^_`{|}~]+'

In [83]:
pured_split_tokens = []
for sent in split_tokens:
    tar=[word for word in sent if re.sub(r1, '', word)==word and len(word)>2]
    if tar:
        pured_split_tokens.append(tar)

In [84]:
# 生成LDA模型
def LDA_model(words_list):
    # 构造词典
    # Dictionary()方法遍历所有的文本，为每个不重复的单词分配一个单独的整数ID，同时收集该单词出现次数以及相关的统计信息
    dictionary = corpora.Dictionary(words_list)
#     print(dictionary)
#     print('打印查看每个单词的id:')
#     print(dictionary.token2id)  # 打印查看每个单词的id
 
    # 将dictionary转化为一个词袋
    # doc2bow()方法将dictionary转化为一个词袋。得到的结果corpus是一个向量的列表，向量的个数就是文档数。
    # 在每个文档向量中都包含一系列元组,元组的形式是（单词 ID，词频）
    corpus = [dictionary.doc2bow(words) for words in words_list]
#     print('输出每个文档的向量:')
#     print(corpus)  # 输出每个文档的向量
 
    # LDA主题模型
    # num_topics -- 必须，要生成的主题个数。
    # id2word    -- 必须，LdaModel类要求我们之前的dictionary把id都映射成为字符串。
    # passes     -- 可选，模型遍历语料库的次数。遍历的次数越多，模型越精确。但是对于非常大的语料库，遍历太多次会花费很长的时间。
    lda_model = models.ldamodel.LdaModel(corpus=corpus, num_topics=2, id2word=dictionary, passes=10)
 
    return lda_model

In [66]:
# 获取训练后的LDA模型
lda_model = LDA_model(pured_split_tokens)

# 可以用 print_topic 和 print_topics 方法来查看主题
# 打印所有主题，每个主题显示5个词
topic_words = lda_model.print_topics(num_topics=5, num_words=5)
print('打印所有主题，每个主题显示5个词:')
print(topic_words)

# 输出该主题的的词及其词的权重
words_list = lda_model.show_topic(0, 20)
print('输出该主题的的词及其词的权重:')
print(words_list)

打印所有主题，每个主题显示5个词:
[(0, '0.048*"mom" + 0.035*"school" + 0.034*"case" + 0.022*"meeting" + 0.021*"week"'), (1, '0.032*"time" + 0.020*"family" + 0.017*"court" + 0.014*"home" + 0.013*"thing"')]
输出该主题的的词及其词的权重:
[('mom', 0.048121765), ('school', 0.03487527), ('case', 0.034221936), ('meeting', 0.022419142), ('week', 0.021112563), ('phone', 0.02036473), ('date', 0.017823054), ('message', 0.017481834), ('email', 0.016758647), ('issue', 0.013176496), ('report', 0.012495851), ('day', 0.012445006), ('police', 0.011745162), ('point', 0.008995797), ('note', 0.008931411), ('cell', 0.008251044), ('work', 0.007342673), ('office', 0.0072296956), ('staff', 0.0068044835), ('session', 0.0064216596)]


# MLE

In [41]:
import json
import tweepy
import re
import time
import os
import nltk
from nltk.util import everygrams
from nltk.lm.preprocessing import pad_both_ends
from nltk.lm.preprocessing import padded_everygram_pipeline
from nltk.lm import MLE, KneserNeyInterpolated
from nltk import word_tokenize, sent_tokenize 
from nltk.lm.preprocessing import padded_everygram_pipeline
import numpy as np

In [42]:
trigram_train_data, trigram_padded_sents = padded_everygram_pipeline(3, pured_split_tokens)
trigram = MLE(3)
trigram.fit(trigram_train_data, trigram_padded_sents)

In [43]:
# reference: https://www.kaggle.com/alvations/n-gram-language-model-with-nltk
from nltk.tokenize.treebank import TreebankWordDetokenizer

detokenize = TreebankWordDetokenizer().detokenize

def generate_sent(model, num_words, random_seed=7):
    """
    :param model: An ngram language model from `nltk.lm.model`.
    :param num_words: Max no. of words to generate.
    :param random_seed: Seed value for random.
    """
    content = []
    for token in model.generate(num_words, random_seed=random_seed):
        if token == '<s>':
            continue
        if token == '</s>':
            break
        content.append(token)
    return detokenize(content)

In [44]:
#generate sentence
for i in range(10):
    print(generate_sent(trigram, 20, random_seed=i))

reveal tell desmond desmond want ask visit bail raise

unit wednesday ice cube cafeteria worker woman
attach correspondence expert surveillance datum
house
goal set meeting mom ready school
phone revocation attorney
present
typically meet danequa family
bjc clerk office require fund request


# Pure Frequency

In [45]:
D=dict()
for sent in pured_split_tokens:
    for word in sent:
        if (ord("a")<ord(word[0])<=ord("z") or ord("A")<ord(word[0])<=ord("Z")):
            if word not in D:
                D[word]=1
            else:
                D[word]+=1
DL=sorted(list(D.items()),key=lambda x:-x[1])
print("Most 50 words in POS, ",[DL[i][0] for i in range(50)])

Most 50 words in POS,  ['desmond', 'talk', 'school', 'mom', 'want', 'tell', 'know', 'need', 'case', 'work', 'time', 'court', 'come', 'like', 'home', 'day', 'report', 'think', 'week', 'dys', 'leave', 'try', 'email', 'note', 'good', 'phone', 'let', 'date', 'police', 'check', 'issue', 'hear', 'send', 'ziyad', 'look', 'house', 'speak', 'family', 'help', 'meet', 'feel', 'today', 'meeting', 'thing', 'kid', 'start', 'tomorrow', 'explain', 'sure', 'new']


In [111]:
print("Most 50 nouns in POS, ",[DL[i][0] for i in range(100) if nltk.pos_tag([DL[i][0]])[0][1]=="NN"])

Most 50 nouns in POS,  ['desmond', 'talk', 'school', 'mom', 'want', 'tell', 'need', 'case', 'work', 'time', 'court', 'home', 'day', 'report', 'think', 'week', 'dys', 'try', 'email', 'note', 'phone', 'date', 'check', 'issue', 'hear', 'send', 'ziyad', 'look', 'house', 'speak', 'family', 'help', 'meet', 'feel', 'today', 'meeting', 'thing', 'kid', 'start', 'tomorrow', 'explain', 'sure', 'probation', 'thank', 'charge', 'officer', 'visit', 'message', 'hour', 'community', 'mother', 'use', 'father', 'schedule', 'jaden', 'record', 'bail', 'motion', 'maureen', 'danielle', 'month', 'point', 'peter', 'confirm', 'stay', 'update', 'hope', 'bring', 'pass', 'file', 'request', 'end', 'plan', 'window', 'release', 'office', 'cop', 'reach', 'year', 'judge', 'gps', 'boston', 'class', 'hearing', 'friend', 'clinician', 'write', 'remind']


In [104]:
nltk.pos_tag(["fee"])

[('tell', 'NN')]

In [113]:
print("Most 50 verbs in POS, ",[DL[i][0] for i in range(100) if nltk.pos_tag([DL[i][0]])[0][1]=="VB"])

Most 50 nouns in POS,  ['know', 'come', 'leave', 'let', 'find', 'happen']


In [114]:
print("Most 50 JJ in POS, ",[DL[i][0] for i in range(100) if nltk.pos_tag([DL[i][0]])[0][1]=="JJ"])

Most 50 nouns in POS,  ['good', 'new', 'open']


In [112]:
print("Most 50 IN in POS, ",[DL[i][0] for i in range(100) if nltk.pos_tag([DL[i][0]])[0][1]=="IN"])

Most 50 nouns in POS,  ['like']
